In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import numpy as np

import simpy
import simpy.events as evt

In [2]:
dt = pd.DataFrame(columns=['Line','From','To','Running_Time','Distance','Pre_signal','Signal_Block'])

In [3]:
dt['Line'] = ['HS2','HS2','HS2']
dt['From'] = ['London Euston','London Old Oak Common ','Birmingham Interchange']
dt['To'] = ['London Old Oak Common ','Birmingham Interchange','Birmingham Curzon Street']
dt['Running_Time'] = [5,31,9]
dt['Distance']= [25,145,45]
dt['Pre_signal']= [1,5,1]
dt['Signal_Block']= [1,5,1]

A-10-15-B-10-15-10-15-10-15-10-15-10-15-C-10-15-D

In [4]:
dt

,Line,From,To,Running_Time,Distance,Pre_signal,Signal_Block
0,HS2,London Euston,London Old Oak Common,5,25,1,1
1,HS2,London Old Oak Common,Birmingham Interchange,31,145,5,5
2,HS2,Birmingham Interchange,Birmingham Curzon Street,9,45,1,1


In [5]:
dt['Running_Time']=dt['Running_Time'].apply(lambda x: int(x*60))

dt['Distance']=dt['Distance'].apply(lambda x: int(x*1000))

In [6]:
dt

,Line,From,To,Running_Time,Distance,Pre_signal,Signal_Block
0,HS2,London Euston,London Old Oak Common,300,25000,1,1
1,HS2,London Old Oak Common,Birmingham Interchange,1860,145000,5,5
2,HS2,Birmingham Interchange,Birmingham Curzon Street,540,45000,1,1


Physics Calculation: time to travel a given distance based on acceleration and max velocity

In [7]:
def timeTo(A, maxV, d):
    """
    A       constant acceleration, m/s²
    maxV    maximumum velocity, m/s
    d       distance, km
    return  time in seconds required to travel
    """
    tA = maxV/A          # time to accelerate to maxV
    dA = A*tA*tA                # distance traveled during acceleration from 0 to maxV and back to 0
    if (d < dA):                # train never reaches full speed?
        return np.sqrt(4.0*d/A)        # time needed to accelerate to half-way point then decelerate to destination
    else:
        return 2*tA + (d-dA)/maxV   # time to accelerate to maxV plus travel at maxV plus decelerate to destination

Calculate acceleration under the assumption that there is no effective max velocity, i.e. the train is accelerating upto half-way point and then decelerating. For short distances this should be accurate, for long distances this should result in an underestimate.

In [8]:
for i in range(len(dt)):
    t1=dt.at[i,'Running_Time']
    dt.at[i, 't']=t1
                           
dt['a']=4*dt['Distance']/dt['t']**2

Solve $t=\frac{v_{max}}{a}+\frac{d}{v_{max}}$ for $v_{max}$ means solving $v_{max}^2-atv_{max}+ad=0$ which results in $v_{max}=\frac{at}{2} - \sqrt{\frac{a^2t^2}{4}-ad}$. 

The condition $\frac{a^2t^2}{4}-ad<0$ corresponds to the situation that the train never reaches on this seqment its maximum speed, in which case we revert to $v_{max}=\frac{at}{2}$.

In [9]:
dt['v']=None
for i in range(len(dt)):
    t=dt.at[i,'t']
    d=dt.at[i,'Distance']
    a=dt['a'].max()
    q=t**2*a**2/4-a*d
    if q<0:
        # corresponds to the case that the train never reaches v_max
        # it has to start decelerate half way through
        q=0
    dt.at[i, 'v']=t*a/2-np.sqrt(q)

In [10]:
for i in range(len(dt)):
    a = dt['a'].max()
    v = dt['v'].max()
    d = dt.at[i, 'Distance']
    dt.at[i, 'Drive_Time']=int(timeTo(a, v, d))

In [11]:
dt

,Line,From,To,Running_Time,Distance,Pre_signal,Signal_Block,t,a,v,Drive_Time
0,HS2,London Euston,London Old Oak Common,300,25000,1,1,300.0,1.111111,166.667,300.0
1,HS2,London Old Oak Common,Birmingham Interchange,1860,145000,5,5,1860.0,0.167649,81.1429,1020.0
2,HS2,Birmingham Interchange,Birmingham Curzon Street,540,45000,1,1,540.0,0.617284,100,420.0


In [16]:
class Train(object):  
    def __init__(self, i, line, route):
        self.name = line+'- [Train '+f"{i:2d}"+']'
        self.line=line
        self.route = route
        self.p     = 0
    
    def process(self):
        here = self.route[0]   # starting location
        for dest in self.route[1:]:
            data=dt[dt['Line']==self.line][dt['From']==here]
            drivetime=data.iloc[0].at['Drive_Time']
            yield env.timeout(drivetime)
            print(f"{now():s} {self.name:s} dep {here:s} for {dest:s}")
            here=dest
            yield env.timeout(drivetime)
            print(f"{now():s} {self.name:s} arr {here:s}")



In [17]:
def line(name='HS2', start=6*3600, stop=7*3600, timing=300):
    data=dt[dt['Line']==name]
    stations=data['From'].to_list()
    stations+=[data['To'].to_list()[-1]]
    yield env.timeout(start-env.now) # the line starts operating at 6am
    for i in range(int((stop-start)/timing)):
        t=Train(i, name, stations)
        env.process(t.process())
        yield env.timeout(timing)

In [18]:
def daytime(t):
    t=int(t)
    return f"{t//3600:02d}:{(t%3600)//60:02d}:{t%60:02d}"
def now():
    return daytime(env.now)

In [19]:
env = simpy.Environment()

env.process(line())
env.run()

06:05:00 HS2- [Train  0] dep London Euston for London Old Oak Common 
06:10:00 HS2- [Train  1] dep London Euston for London Old Oak Common 
06:10:00 HS2- [Train  0] arr London Old Oak Common 
06:15:00 HS2- [Train  2] dep London Euston for London Old Oak Common 
06:15:00 HS2- [Train  1] arr London Old Oak Common 
06:20:00 HS2- [Train  3] dep London Euston for London Old Oak Common 
06:20:00 HS2- [Train  2] arr London Old Oak Common 
06:25:00 HS2- [Train  4] dep London Euston for London Old Oak Common 
06:25:00 HS2- [Train  3] arr London Old Oak Common 
06:27:00 HS2- [Train  0] dep London Old Oak Common  for Birmingham Interchange
06:30:00 HS2- [Train  5] dep London Euston for London Old Oak Common 
06:30:00 HS2- [Train  4] arr London Old Oak Common 
06:32:00 HS2- [Train  1] dep London Old Oak Common  for Birmingham Interchange
06:35:00 HS2- [Train  6] dep London Euston for London Old Oak Common 
06:35:00 HS2- [Train  5] arr London Old Oak Common 
06:37:00 HS2- [Train  2] dep London Old 

In [44]:
class Train(object):  
    def __init__(self, i, bcs, line, route):
        self.name = line+'- [Train '+f"{i:2d}"+']'
        self.line=line
        self.route = route
        self.p     = 0
        bcs = bcs
    
    def process(self):
        here = self.route[0]   # starting location
        for dest in self.route[1:]:
            data=dt[dt['Line']==self.line][dt['From']==here]
            drivetime=data['Drive_Time'].item()
            ps = data['Pre_signal'].item()
            sb = data['Signal_Block'].item()
            yield env.timeout(drivetime)
            print(f"{now():s} {self.name:s} dep {here:s} for {dest:s}")
            here=dest
            with bcs.request() as req:
                yield req
                print(f"{self.name:s} train entering in block")
                dwelltime = 360
                yield env.timeout(dwelltime)
                print(f"{now():s} {self.name:s} arr {here:s}")

In [54]:
def line(bcs, name='HS2', start=6*3600, stop=7*3600, timing=1200):
    data=dt[dt['Line']==name]
    stations=data['From'].to_list()
    stations+=[data['To'].to_list()[-1]]
    yield env.timeout(start-env.now) # the line starts operating at 6am
    for i in range(int((stop-start)/timing)):
        t=Train(i, bcs, name, stations)
        env.process(t.process())
        yield env.timeout(timing)

In [55]:
def daytime(t):
    t=int(t)
    return f"{t//3600:02d}:{(t%3600)//60:02d}:{t%60:02d}"
def now():
    return daytime(env.now)

In [56]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=1)
env.process(line(bcs))
env.run()

06:05:00 HS2- [Train  0] dep London Euston for London Old Oak Common 
HS2- [Train  0] train entering in block
06:11:00 HS2- [Train  0] arr London Old Oak Common 
06:25:00 HS2- [Train  1] dep London Euston for London Old Oak Common 
HS2- [Train  1] train entering in block
06:28:00 HS2- [Train  0] dep London Old Oak Common  for Birmingham Interchange
06:31:00 HS2- [Train  1] arr London Old Oak Common 
HS2- [Train  0] train entering in block
06:37:00 HS2- [Train  0] arr Birmingham Interchange
06:44:00 HS2- [Train  0] dep Birmingham Interchange for Birmingham Curzon Street
HS2- [Train  0] train entering in block
06:45:00 HS2- [Train  2] dep London Euston for London Old Oak Common 
06:48:00 HS2- [Train  1] dep London Old Oak Common  for Birmingham Interchange
06:50:00 HS2- [Train  0] arr Birmingham Curzon Street
HS2- [Train  2] train entering in block
06:56:00 HS2- [Train  2] arr London Old Oak Common 
HS2- [Train  1] train entering in block
07:02:00 HS2- [Train  1] arr Birmingham Interchan

In [57]:
test = pd.DataFrame(columns=['Line','From','To','Running_Time','Distance','type'])

In [59]:
test['From']=['A/PS1','SB1','B/PS2','SB2','PS3','SB3']
test['To']=['SB1','B','SB2','PS2','SB3','C']
test['Line']=['HS2','HS2','HS2','HS2','HS2','HS2']
test['type']=['PS','SB','PS','SB','PS','SB']
test['Distance']=[10000,15000,10000,15000,10000,15000]
test['Running_Time']=[300,420,360,480,300,540]

In [60]:
test

,Line,From,To,Running_Time,Distance,type
0,HS2,A/PS1,SB1,300,10000,PS
1,HS2,SB1,B,420,15000,SB
2,HS2,B/PS2,SB2,360,10000,PS
3,HS2,SB2,PS2,480,15000,SB
4,HS2,PS3,SB3,300,10000,PS
5,HS2,SB3,C,540,15000,SB


In [112]:
class Train(object):  
    def __init__(self, i, bcs, line, route):
        self.name = line+'- [Train '+f"{i:2d}"+']'
        self.line=line
        self.route = route
        self.p     = 0
        bcs = bcs
    
    def process(self):
        here = self.route[0]   # starting location
        for dest in self.route[1:]:
            data=dt[dt['Line']==self.line][dt['From']==here]
            drivetime=data['Running_Time'].item()
            blocktype = data['Type'].item()
            if here == 'London Euston' or here == 'London Old Oak Common' or here == 'Birmingham Interchange' or here == 'Birmingham Curzon Street':
                yield env.timeout(drivetime)
            print(f"{now():s} {self.name:s} dep {here:s} for {dest:s}")
            here=dest
            signal = 'green'
            
            if signal == 'green':
                with bcs.request() as req:
                    yield req
                    signal = 'red'
                    if blocktype == 'PS':
                        print(f"{self.name:s} train entering in pre signal block and pre signal is ",signal)
                    else:
                        print(f"{self.name:s} train entering in signal block and signal is ",signal)
                    yield env.timeout(drivetime)
                    print(f"{now():s} {self.name:s} arr {here:s}")
                    yield env.timeout(5)
                    signal = 'green'
                    if blocktype == 'PS':
                        print(f"{now():s} pre signal is ",signal)
                    else:
                        print(f"{now():s} signal is ",signal)

In [113]:
def line(bcs, name='HS2', start=6*3600, stop=7*3600, timing=1200):
    data=test[test['Line']==name]
    stations=data['From'].to_list()
    stations+=[data['To'].to_list()[-1]]
    yield env.timeout(start-env.now) # the line starts operating at 6am
    for i in range(int((stop-start)/timing)):
        t=Train(i, bcs, name, stations)
        env.process(t.process())
        yield env.timeout(timing)

In [114]:
def daytime(t):
    t=int(t)
    return f"{t//3600:02d}:{(t%3600)//60:02d}:{t%60:02d}"
def now():
    return daytime(env.now)

In [115]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=1)
env.process(line(bcs))
env.run()

06:05:00 HS2- [Train  0] dep A/PS1 for SB1
HS2- [Train  0] train entering in pre signal block and pre signal is  red
06:10:00 HS2- [Train  0] arr SB1
06:10:05 pre signal is  green
06:17:05 HS2- [Train  0] dep SB1 for B/PS2
HS2- [Train  0] train entering in signal block and signal is  red
06:24:05 HS2- [Train  0] arr B/PS2
06:24:10 signal is  green
06:25:00 HS2- [Train  1] dep A/PS1 for SB1
HS2- [Train  1] train entering in pre signal block and pre signal is  red
06:30:00 HS2- [Train  1] arr SB1
06:30:05 pre signal is  green
06:30:10 HS2- [Train  0] dep B/PS2 for SB2
HS2- [Train  0] train entering in pre signal block and pre signal is  red
06:36:10 HS2- [Train  0] arr SB2
06:36:15 pre signal is  green
06:37:05 HS2- [Train  1] dep SB1 for B/PS2
HS2- [Train  1] train entering in signal block and signal is  red
06:44:05 HS2- [Train  1] arr B/PS2
06:44:10 signal is  green
06:44:15 HS2- [Train  0] dep SB2 for PS3
HS2- [Train  0] train entering in signal block and signal is  red
06:45:00 HS2-